In [1]:
import zipfile
from os import listdir
from os.path import isfile, join
from pathlib import Path

from tqdm import tqdm

In [2]:
levelFolder = Path("levels/")

In [3]:
levelsDownloaded = [f for f in listdir(levelFolder) if isfile(join(levelFolder, f))]

In [4]:
archive = zipfile.ZipFile(join(levelFolder, '007_-_bond_mario_bond__2958.zip'), 'r')
leveldata = archive.read('course_data_000.bcd')

In [5]:
leveldata[15215]

96

In [6]:
with open('tools/tmp/course_data_000.bcd', 'wb') as f:
    f.write(leveldata)

In [7]:
!.\tools\smm2dec.exe -h .\tools\tmp\course_data_000.bcd .\tools\tmp\course_data_dec.bcd

Decrypting course .\tools\tmp\course_data_000.bcd to .\tools\tmp\course_data_dec.bcd...
Done!


In [8]:
with open('tools/tmp/course_data_dec.bcd', 'rb') as f:
    leveldata = f.read()

In [9]:
leveldata[15215]

0

In [12]:
AESIV = leveldata[:16]  #array
randomiserState = leveldata[17:33]
AESCMAC = leveldata[34:50]


gameVersion = leveldata[:4]
gameVersion2 = leveldata[4:6]
padding = leveldata[6:8]
CRC32 = leveldata[8:12]
SCDL = leveldata[12:16]
header = leveldata[16:528]
mainLevelArea = leveldata[528:12285]
subLevelArea = leveldata[12285:24042]
#manca la parte cripto

In [14]:
startY = header[0]
endY = header[1]
goalXx100 = header[2:4]
timeLimit = header[4:6]
targetAmountClearCondition = header[6:8]
year = header[8:10]
month = header[10]
day = header[11]
hour = header[12]
minute = header[13]
unknow = header[14]
clearConditionType = header[15]
clearConditionObject = header[16:20]
unknow4 = header[20:24]
levelFlags = header[24:28]
clearCheckTime = header[28:32]
randomValue = header[32:36]
courseWorld = header[36:40]

unknow2 = header[240]
gameStyle = header[241:244]
courseName = header[244:277]
courseDescription = header[277:378]

In [108]:
len(header)

512

In [110]:
int.from_bytes(year,"little", signed = False)

2019

In [75]:
gameVersion2

b'\x10\x00'

In [114]:
month

7

In [17]:
courseDescription.decode("utf-8")

'\x00 \x00B\x00o\x00n\x00d\x00 \x00j&\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00M\x00u\x00s\x00i\x00c\x00 \x00a\x00n\x00d\x00 \x00P\x00l\x00a\x00y\x00!\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'